In [ ]:
import yfinance as yf
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt

def garman_klass_volatility(price_data, window=20, trading_periods=252):
    """
    Calculate Garman-Klass volatility using OHLC price data.
    
    Parameters:
    price_data: DataFrame with columns ['Open', 'High', 'Low', 'Close']
    window: Rolling window size for volatility calculation (default: 20 days)
    trading_periods: Number of trading periods in a year (default: 252 for daily data)
    
    Returns:
    pandas Series with annualized Garman-Klass volatility
    """
    # Calculate log ratios
    log_hl = np.log(price_data['High'] / price_data['Low'])
    log_co = np.log(price_data['Close'] / price_data['Open'])
    
    # Apply Garman-Klass formula
    rs = 0.5 * log_hl**2 - (2 * math.log(2) - 1) * log_co**2
    
    # Calculate rolling variance and annualize
    rolling_var = rs.rolling(window=window).mean()
    gk_volatility = np.sqrt(trading_periods * rolling_var)
    
    return gk_volatility


# def Yang_Zhang_RV_own_data(data):
#     # importing needed libraries
#     import pandas as pd
#     import numpy as np
#     import warnings
#     warnings.filterwarnings("ignore")
    
#     # Yang_Zhang_RV formula is give as:
#     # RV^2 = Vo + k*Vc + (1-k)*Vrs
#     # where Vo = 1/(n-1)*sum(Oi-Obar)^2
#     # with oi = normalized opening price at time t and Obar = mean of normalized opening prices
#     # Vc = = 1/(n-1)*sum(ci-Cbar)^2
#     # with ci = normalized close price at time t and Cbar = mean of normalized close prices
#     # k = 0.34/(1.34+(n+1)/(n-1))
#     # with n = total number of days or time periods considered
#     # Vrs (Rogers & Satchell RV proxy) = ui(ui-ci)+di(di-ci)
#     # with ui = ln(Hi/Oi), ci = ln(Ci/Oi), di=(Li/Oi), oi = ln(Oi/Ci-1)
#     # where Hi = high price at time t and Li = low price at time t
    
#     data["ui"]=np.log(np.divide(data["High"][1:],data["Open"][1:]))
#     data["ci"]=np.log(np.divide(data["Close"][1:],data["Open"][1:]))
#     data["di"]=np.log(np.divide(data["Low"][1:],data["Open"][1:]))
#     data["oi"]=np.log(np.divide(data["Open"][1:],data["Close"][:len(data)-1]))
#     data=data[1:]
#     data["RS"]=data["ui"]*(data["ui"]-data["ci"])+data["di"]*(data["di"]-data["ci"])
#     RS_var= data["RS"].groupby(pd.Grouper(freq='1D')).mean().dropna()
#     Vc_and_Vo=data[["oi", "ci"]].groupby(pd.Grouper(freq='1D')).var().dropna()
#     n=int(len(data)/len(RS_var))
#     k = 0.34/(1.34+(n+1)/(n-1))
#     Yang_Zhang_RV=np.sqrt((1-k)*RS_var+Vc_and_Vo["oi"]+Vc_and_Vo["ci"]*k)
#     Yang_Zhang_RV_df=pd.DataFrame(Yang_Zhang_RV)
#     Yang_Zhang_RV_df.rename(columns={0: "Yang & Zhang RV proxy"},inplace=True)
    
#     return Yang_Zhang_RV_df




In [ ]:
def yang_zhang_volatility(price_data, window=20, trading_periods=252):
    """
    Calculate Yang-Zhang volatility using OHLC price data (corrected formula).
    
    Parameters:
    price_data: DataFrame with columns ['Open', 'High', 'Low', 'Close']
    window: Rolling window size for volatility calculation
    trading_periods: Annualization factor (default: 252 for daily)
    
    Returns:
    pandas Series with annualized Yang-Zhang volatility
    """
    # Calculate log returns
    log_ho = np.log(price_data['High'] / price_data['Open'])
    log_lo = np.log(price_data['Low'] / price_data['Open'])
    log_co = np.log(price_data['Close'] / price_data['Open'])
    log_oc = np.log(price_data['Open'] / price_data['Close'].shift(1))

    # Rogers-Satchell (RS) volatility proxy
    rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)
    # Vo: variance of overnight returns
    vo = log_oc.rolling(window=window).var()
    # Vc: variance of open-to-close returns
    vc = log_co.rolling(window=window).var()
    # k parameter
    k = 0.34 / (1.34 + (window + 1) / (window - 1))
    # Vrs: mean of RS volatility
    vrs = rs.rolling(window=window).mean()
    # Yang-Zhang volatility
    yz_var = vo + k * vc + (1 - k) * vrs
    yz_vol = np.sqrt(yz_var * trading_periods)
    return yz_vol